In [ ]:
# tests/test_fabricate_vectors.py
import numpy as np
import tensorflow as tf
from IEModules.Helper_Functions import fabricate_vectors_for_f1
from IEModules.Custom_Metrics import CustomNoBackgroundF1Score

@pytest.mark.parametrize("target, with_bg", [
    (0.30, True),
    (0.55, True),
    (0.87, True),
    (0.20, False),
    (0.77, False),
    (1.00, False),
])
def test_fabricated_f1(target, with_bg):
    y_true, y_pred = fabricate_vectors_for_f1(target,
                                              length=200,
                                              include_background=with_bg,
                                              seed=42)
    metric = CustomNoBackgroundF1Score(
        num_classes=5,   # always 5, the metric strips bg if present
        name="nb_f1"
    )
    metric.update_state(tf.constant(y_true), tf.constant(y_pred))
    realised = metric.result().numpy()
    # allow tiny float error
    assert np.isclose(realised, target, atol=1e-6), \
        f"wanted {target}, got {realised}"


In [ ]:
from IEModules.Helper_Functions import fabricate_vectors_for_f1
from IEModules.Custom_Metrics   import CustomNoBackgroundF1Score
import tensorflow as tf

yt, yp = fabricate_vectors_for_f1(0.77, 200, include_background=True, seed=1)
metric = CustomNoBackgroundF1Score(num_classes=5)
metric.update_state(tf.constant(yt), tf.constant(yp))
print("Realised F1 =", round(metric.result().numpy(), 4))
